In [1]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Spark SQL") \
    .getOrCreate()

In [3]:
# Read the CSV files into a DataFrame accounts, country_abbreviation, transactions
df_accounts = spark.read.csv("accounts.csv", header=True, inferSchema=True, sep=";")
df_country_abbreviation = spark.read.csv("country_abbreviation.csv", header=True, inferSchema=True, sep=";")
df_transactions = spark.read.csv("transactions.csv", header=True, inferSchema=True, sep=";")


In [29]:
# Create temporary view on df
df_accounts.createOrReplaceTempView("accounts")
df_transactions.createOrReplaceTempView("transactions")
result_df_accounts = spark.sql("""
    SELECT
        COALESCE(t2.account_type, 'Without type') account_type
        ,CAST(count(*) as INTEGER) account_type_count
    FROM 
    (
        SELECT DISTINCT a.id, t.account_type
        FROM accounts a left JOIN transactions t on t.id = a.id
    ) t2
    GROUP BY t2.account_type
""")
result_df_accounts.show()
result_df_accounts.printSchema()
#df_transactions.createOrReplaceTempView("transactions")



+------------+------------------+
|account_type|account_type_count|
+------------+------------------+
|    Personal|            481997|
|Without type|                20|
|Professional|            482170|
|    Business|            482350|
+------------+------------------+

root
 |-- account_type: string (nullable = false)
 |-- account_type_count: integer (nullable = false)



In [38]:
# return transaction balance datast
result_df_transaction_balance = spark.sql("""
    SELECT CAST(a.id AS VARCHAR(38)) account_id
        ,CAST(COALESCE(SUM(t.amount), 0) AS VARCHAR(38)) as balance
        ,MAX(t.transaction_date) latest_date
    FROM accounts a
        LEFT JOIN transactions t on a.id = t.id
    --WHERE t.id is null
    GROUP BY a.id
    
""")
result_df_transaction_balance.show()
result_df_transaction_balance.printSchema()

+----------+------------------+-----------+
|account_id|           balance|latest_date|
+----------+------------------+-----------+
|        12|32998.329999999994| 2021-05-16|
|        22|          60841.95| 2021-12-05|
|        26| 50989.13999999999| 2020-10-26|
|        27|16375.809999999998| 2021-08-25|
|        28|53104.520000000004| 2021-11-13|
|        31|          26232.62| 2021-11-02|
|        34|          22582.96| 2021-04-29|
|        44|          41494.13| 2021-06-02|
|        53|           14782.9| 2020-08-23|
|        65| 47483.56999999999| 2021-10-30|
|        76| 56286.51999999999| 2021-01-21|
|        78|          38812.24| 2020-08-19|
|        81| 63034.61000000001| 2021-11-30|
|        85|          29603.43| 2020-07-05|
|        91| 87256.99999999999| 2021-07-04|
|        93|          11367.93| 2020-07-18|
|       101|52479.630000000005| 2021-08-13|
|       103|          76365.61| 2019-11-15|
|       108|          54106.94| 2020-02-16|
|       115|          52005.16| 